## Second Order Graph Creation (Incorrect)

### Node creation

create a second order node from Temporal graph's edges

In [ ]:
s_nodes = edges.with_columns([
    pl.concat_str([pl.col("v_s"), pl.col("v_d")], separator="->").alias("node_id"),
    pl.col("v_s").alias("f_i"),
    pl.col("v_d").alias("b_i"),
]).select(["node_id", "f_i", "b_i"])

In [ ]:
s_nodes.collect().head()

node_id,f_i,b_i
str,u32,u32
"""15720->1""",15720,1
"""17905->1""",17905,1
"""50608->1""",50608,1
"""77968->1""",77968,1
"""78253->1""",78253,1


### Edge creation

In [ ]:
s_edges = s_nodes.join(
    s_nodes,
    left_on="b_i", 
    right_on="f_i",
    suffix="_d",
    how="inner"
).rename({"node_id": "v_s", "node_id_d": "v_d"})

In [ ]:
s_edges.collect().head()

v_s,f_i,b_i,v_d,b_i_d
str,u32,u32,str,u32
"""81923->404989""",81923,404989,"""404989->60""",60
"""433814->404989""",433814,404989,"""404989->60""",60
"""372745->134996""",372745,134996,"""134996->62""",62
"""90193->347236""",90193,347236,"""347236->64""",64
"""417336->19969""",417336,19969,"""19969->66""",66


### Weight Calculation

In [ ]:
# Count times where one node goes to another
s_edges_with_count = s_edges.group_by(["v_s", "v_d"]).agg([
    pl.len().alias("spec_count"),
    pl.col("f_i").first(),
    pl.col("b_i").first(),
    pl.col("b_i_d").first()
])

denom_P = s_edges.group_by("v_s").agg([
    pl.len().alias("s_to_any")
])

denom_P_prime = s_edges.group_by("v_d").agg([
    pl.len().alias("any_to_d")
])

In [ ]:
print(s_edges_with_count.collect().head())

shape: (5, 6)
┌────────────────┬────────────────┬────────────┬────────┬────────┬────────┐
│ v_s            ┆ v_d            ┆ spec_count ┆ f_i    ┆ b_i    ┆ b_i_d  │
│ ---            ┆ ---            ┆ ---        ┆ ---    ┆ ---    ┆ ---    │
│ str            ┆ str            ┆ u32        ┆ u32    ┆ u32    ┆ u32    │
╞════════════════╪════════════════╪════════════╪════════╪════════╪════════╡
│ 358739->165769 ┆ 165769->287998 ┆ 1          ┆ 358739 ┆ 165769 ┆ 287998 │
│ 273792->284679 ┆ 284679->450261 ┆ 1          ┆ 273792 ┆ 284679 ┆ 450261 │
│ 230114->241882 ┆ 241882->264112 ┆ 1          ┆ 230114 ┆ 241882 ┆ 264112 │
│ 323753->206126 ┆ 206126->147433 ┆ 1          ┆ 323753 ┆ 206126 ┆ 147433 │
│ 451074->67350  ┆ 67350->155525  ┆ 1          ┆ 451074 ┆ 67350  ┆ 155525 │
└────────────────┴────────────────┴────────────┴────────┴────────┴────────┘


In [ ]:
print(denom_P.collect().head())

print(denom_P_prime.collect().head())

shape: (5, 2)
┌────────────────┬──────────┐
│ v_s            ┆ s_to_any │
│ ---            ┆ ---      │
│ str            ┆ u32      │
╞════════════════╪══════════╡
│ 372412->37423  ┆ 1        │
│ 127780->332423 ┆ 2        │
│ 122044->377464 ┆ 1        │
│ 193253->209210 ┆ 1        │
│ 101045->331989 ┆ 2        │
└────────────────┴──────────┘
shape: (5, 2)
┌────────────────┬──────────┐
│ v_d            ┆ any_to_d │
│ ---            ┆ ---      │
│ str            ┆ u32      │
╞════════════════╪══════════╡
│ 84083->59771   ┆ 1        │
│ 403849->319233 ┆ 1        │
│ 405261->125954 ┆ 3        │
│ 213058->213532 ┆ 1        │
│ 131616->383677 ┆ 1        │
└────────────────┴──────────┘


In [ ]:
s_edges_with_weight = s_edges_with_count.join(
    denom_P,
    on="v_s",
    how="left"
).join(
    denom_P_prime,
    on="v_d",
    how="left"
).with_columns([
    (pl.col("spec_count") / pl.col("s_to_any")).alias("P"),
    (pl.col("spec_count") / pl.col("any_to_d")).alias("P_prime")
]).with_columns([
    (pl.max_horizontal([pl.col("P"), pl.col("P_prime")])).alias("weight")
]).select([
    "v_s", "v_d", "spec_count", "s_to_any", "any_to_d", "P", "P_prime", "weight"
])

In [ ]:
print(s_edges_with_weight.collect().head())

shape: (5, 8)
┌────────────────┬────────────────┬────────────┬──────────┬──────────┬──────────┬─────────┬────────┐
│ v_s            ┆ v_d            ┆ spec_count ┆ s_to_any ┆ any_to_d ┆ P        ┆ P_prime ┆ weight │
│ ---            ┆ ---            ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---     ┆ ---    │
│ str            ┆ str            ┆ u32        ┆ u32      ┆ u32      ┆ f64      ┆ f64     ┆ f64    │
╞════════════════╪════════════════╪════════════╪══════════╪══════════╪══════════╪═════════╪════════╡
│ 199524->285656 ┆ 285656->171847 ┆ 1          ┆ 3        ┆ 1        ┆ 0.333333 ┆ 1.0     ┆ 1.0    │
│ 42842->348638  ┆ 348638->276249 ┆ 1          ┆ 1        ┆ 1        ┆ 1.0      ┆ 1.0     ┆ 1.0    │
│ 161645->293626 ┆ 293626->5445   ┆ 1          ┆ 2        ┆ 50       ┆ 0.5      ┆ 0.02    ┆ 0.5    │
│ 304219->384524 ┆ 384524->298753 ┆ 1          ┆ 2        ┆ 2        ┆ 0.5      ┆ 0.5     ┆ 0.5    │
│ 96223->72686   ┆ 72686->114262  ┆ 1          ┆ 2        ┆ 2        ┆ 0.5   

### Apply second order graph's weight to Temporal graph's edges

In [ ]:
edges_with_weight = edges.with_columns([
    pl.concat_str([pl.col("f_i"), pl.col("b_i")], separator="->").alias("tx_s"),
    pl.concat_str([pl.col("b_i"), pl.col("b_i_d")], separator="->").alias("tx_d")
]).join(
    s_edges_with_weight.select([
        "v_s", "v_d", "weight"
    ]),
    left_on=["tx_s", "tx_d"],
    right_on=["v_s", "v_d"],
    how="left"
)

In [ ]:
edges_with_weight.filter(pl.col("weight").is_null()).collect()

v_s,f_i,b_i,t_i,a_i,Is Laundering,v_d,b_i_d,t_i_d,a_i_d,Is Laundering_d,tx_s,tx_d,weight
u32,str,str,datetime[μs],f64,i64,u32,str,datetime[μs],f64,i64,str,str,f64
15720,"""81269FE50""","""100428660""",2022-09-02 09:51:00,363.25,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0,"""81269FE50->100428660""","""100428660->800C08F10""",null
17905,"""8070EAE50""","""100428660""",2022-09-02 10:00:00,119.65,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0,"""8070EAE50->100428660""","""100428660->800C08F10""",null
50608,"""806018480""","""100428660""",2022-09-02 07:18:00,214.0,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0,"""806018480->100428660""","""100428660->800C08F10""",null
77968,"""80D167C20""","""100428660""",2022-09-02 07:35:00,1394.66,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0,"""80D167C20->100428660""","""100428660->800C08F10""",null
78253,"""80C710A10""","""100428660""",2022-09-02 02:02:00,453.76,0,1,"""800C08F10""",2022-09-02 12:05:00,1000.0,0,"""80C710A10->100428660""","""100428660->800C08F10""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
231812,"""8137529A1""","""813B87B71""",2022-09-06 02:34:00,0.952445,0,459751,"""813B87B71""",2022-09-06 20:48:00,0.026602,0,"""8137529A1->813B87B71""","""813B87B71->813B87B71""",null
217176,"""8093BB640""","""1004286A8""",2022-09-02 00:29:00,871.28,0,459752,"""8144DAE90""",2022-09-02 01:05:00,1614.02,0,"""8093BB640->1004286A8""","""1004286A8->8144DAE90""",null
259236,"""80E2EFCF0""","""1004286A8""",2022-09-02 00:08:00,174.56,0,459752,"""8144DAE90""",2022-09-02 01:05:00,1614.02,0,"""80E2EFCF0->1004286A8""","""1004286A8->8144DAE90""",null
